In [ ]:
import sys
import os
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))
from config import DATA_DIR
from data.loaders.raw_loader import RawEEGLoader
import numpy as np
import mne

os.environ['KMP_DUPLICATE_LIB_OK']='True' # To prevent the kernel from dying.


eeg_path = DATA_DIR / "Sub04" / "karsten.vhdr"
raw = mne.io.read_raw_brainvision(eeg_path, preload=True, verbose=False)

# Check events (from annotations for BrainVision files)
events, event_id = mne.events_from_annotations(raw, verbose=True)

print("Event ID mapping:")
print(event_id)
print("\nFirst 20 events:")
print(events[:20])
print(f"\nTotal events: {len(events)}")
print(f"Unique sample indices: {len(np.unique(events[:, 0]))} / {len(events)}")

# Check for duplicates
sample_indices = events[:, 0]
unique, counts = np.unique(sample_indices, return_counts=True)
duplicates = unique[counts > 1]
if len(duplicates) > 0:
    print(f"\nDuplicate sample indices found: {len(duplicates)}")
    for dup in duplicates[:5]:  # Show first 5
        print(f"  Sample {dup}: {events[events[:, 0] == dup]}")

Used Annotations descriptions: [np.str_('Response/R  4'), np.str_('Response/R 15'), np.str_('Stimulus/S  1'), np.str_('Stimulus/S  2'), np.str_('Stimulus/S  3'), np.str_('Stimulus/S  5'), np.str_('Stimulus/S 14'), np.str_('Stimulus/S 15')]
Event ID mapping:
{np.str_('Response/R  4'): 1004, np.str_('Response/R 15'): 1015, np.str_('Stimulus/S  1'): 1, np.str_('Stimulus/S  2'): 2, np.str_('Stimulus/S  3'): 3, np.str_('Stimulus/S  5'): 5, np.str_('Stimulus/S 14'): 14, np.str_('Stimulus/S 15'): 15}

First 20 events:
[[ 10589      0      1]
 [ 14748      0      2]
 [ 46810      0      5]
 [ 56882      0      1]
 [ 60414      0      2]
 [ 92466      0      5]
 [101346      0      1]
 [104829      0      2]
 [136882      0      5]
 [144477      0      1]
 [148844      0      2]
 [180895      0      5]
 [186808      0      1]
 [190925      0      2]
 [222981      0      5]
 [231207      0      1]
 [234923      0      2]
 [266979      0      5]
 [275355      0      1]
 [279805      0      2]]

T

In [2]:
import sys
import os
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))
from config import DATA_DIR
from data.loaders.raw_loader import RawEEGLoader

os.environ['KMP_DUPLICATE_LIB_OK']='True' # To prevent the kernel from dying.

# Create loader with default config
loader = RawEEGLoader()

# Load raw EEG file
data = loader.load(
      eeg_path= DATA_DIR / "Sub02" / "daniel_1_eeg.vhdr",
      participant_id="sub-002"
  )

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


RuntimeError: Event time samples were not unique. Consider setting the `event_repeated` parameter."

In [ ]:
# Analyze all markers in the recording
import pandas as pd

# Get events from the raw data (using the raw object from earlier)
events, event_id = mne.events_from_annotations(raw, verbose=False)

# Create a reverse mapping from event code to description
event_id_reverse = {v: k for k, v in event_id.items()}

# Count occurrences of each marker
unique_events, counts = np.unique(events[:, 2], return_counts=True)

print("=" * 60)
print("MARKER ANALYSIS")
print("=" * 60)

# Create summary table
summary = []
for event_code, count in zip(unique_events, counts):
    description = event_id_reverse.get(event_code, "Unknown")
    summary.append({
        "Event Code": event_code,
        "Description": description,
        "Count": count
    })

df = pd.DataFrame(summary)
print("\nAll markers found in recording:")
print(df.to_string(index=False))

# Separate stimulus and response markers
print("\n" + "=" * 60)
print("STIMULUS MARKERS (S)")
print("=" * 60)
stimulus_markers = [1, 2, 3, 5, 14, 15]  # Based on event_id mapping
for code in stimulus_markers:
    if code in unique_events:
        desc = event_id_reverse.get(code, "Unknown")
        count = counts[list(unique_events).index(code)]
        print(f"  {desc}: {count}")

print("\n" + "=" * 60)
print("RESPONSE MARKERS (R)")
print("=" * 60)
response_markers = [1004, 1015]  # Based on event_id mapping
for code in response_markers:
    if code in unique_events:
        desc = event_id_reverse.get(code, "Unknown")
        count = counts[list(unique_events).index(code)]
        print(f"  {desc}: {count}")

# Check for markers beyond 1, 2, 5
print("\n" + "=" * 60)
print("MARKERS BEYOND 1, 2, 5")
print("=" * 60)
expected_stimuli = {1, 2, 5}
other_markers = set(unique_events) - expected_stimuli
if other_markers:
    print("Found additional markers:")
    for code in sorted(other_markers):
        desc = event_id_reverse.get(code, "Unknown")
        count = counts[list(unique_events).index(code)]
        print(f"  Code {code} ({desc}): {count} occurrences")
else:
    print("No additional markers found beyond 1, 2, 5")

# Count total stimuli (marker 1 = word start, marker 2 = image start)
print("\n" + "=" * 60)
print("STIMULUS COUNT SUMMARY")
print("=" * 60)
word_count = counts[list(unique_events).index(1)] if 1 in unique_events else 0
image_count = counts[list(unique_events).index(2)] if 2 in unique_events else 0
end_count = counts[list(unique_events).index(5)] if 5 in unique_events else 0

print(f"  Word presentations (S1): {word_count}")
print(f"  Image presentations (S2): {image_count}")
print(f"  Trial ends (S5): {end_count}")
print(f"  Total trials: {end_count}")